In [1]:
import pickle

def save_to_pickle(data,savepath):

    with open(savepath, 'wb') as handle:

        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return None
 
def depickle(savepath):

    with open(savepath, 'rb') as handle:

        b = pickle.load(handle)

    return b
 
import pandas as pd

import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from scipy.stats import genextreme
from scipy.stats import weibull_min
import matplotlib.pyplot as plt

operating_system = 'curnagl'

if operating_system == 'win':
    os.chdir('C:/Users/fabau/OneDrive/Documents/GitHub/master-project-cleaned/')
elif operating_system == 'curnagl':
    os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')
else:
    os.chdir('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/')

# Add the path to the custom library
custom_library_path = os.path.abspath('util/processing/')
sys.path.append(custom_library_path)

import time_series

custom_library_path_2 = os.path.abspath('util/gev')
sys.path.append(custom_library_path_2)

import data_processing#, vgam_loader

In [10]:
# open pickle file

datasets = depickle('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/data/testz/daily_with_storms/climatology_europe_1_2.pkl')
mus_daily_with_storms = []
sigmas_daily_with_storms = []
xi_daily_with_storms = []

cluster_data_original = pd.read_csv('pre_processing/cluster/number_of_clusters/cleaned_cluster_15_v1.csv')
vivian = data_processing.process_daily_climatology('data/time_series_rasters_storms_15h/5_VIVIAN.tif', cluster_data_original, 'VIVIAN')

cluster_number = 3

# Process each dataset
for data in datasets:
    #data_name = data['name'].iloc[0]
    #print("Data:", data_name)

    #print('Cluster number:', cluster_number)
    
    # Group data by cluster and filter for the current cluster
    grouped_data = data.groupby('cluster_n')
    cluster_data = grouped_data.get_group(cluster_number)['wind_speed_None']
    vivian_subset = vivian[vivian['cluster_n'] == cluster_number]['wind_speed_None']

    # Fit the GEV distribution
    shape, loc, scale = weibull_min.fit(cluster_data, method="MLE")

    mus_daily_with_storms.append(loc)
    sigmas_daily_with_storms.append(scale)
    xi_daily_with_storms.append(shape)

KeyError: "No variable named 'cluster_n'. Variables on the dataset include ['i10fg', 'longitude', 'latitude', 'band', 'spatial_ref']"

In [30]:
#datasets[0]['i10fg'].plot()
test_dataframe = time_series.tif_to_dataframe(datasets[0]['i10fg'].squeeze())
peutetre = pd.DataFrame(datasets[0]['i10fg'].squeeze())
import rasterio

# Create a list to store data
data_list = []
transform = '0.25'
nodata_value = np.nan

# Loop through each pixel
for row in range(peutetre.shape[0]):
    for col in range(peutetre.shape[1]):
        # Get the wind speed value
        value = peutetre.iloc[row, col]
        
        # Skip if it's a no-data value
        if value == nodata_value or value is None:
            continue
        
        # Get the coordinates for this pixel
        x, y = rasterio.transform.xy(transform, row, col, offset='center')
        
        # Append the value and coordinates
        data_list.append({f'wind_speed_': value, 'longitude': x, 'latitude': y})

# Convert the list into a DataFrame
df = pd.DataFrame(data_list)

An error occurred: invalid path or file: <xarray.DataArray 'i10fg' (latitude: 153, longitude: 209)> Size: 256kB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * longitude    (longitude) float64 2kB -12.0 -11.75 -11.5 ... 39.5 39.75 40.0
  * latitude     (latitude) float64 1kB 71.0 70.75 70.5 ... 33.5 33.25 33.0
    band         int64 8B 1
    spatial_ref  int64 8B ...


ValueError: GCPTransformer requires sequence of GroundControlPoint